# 1：异常检测与推荐系统
### **by:MLZZY**

### 一：使用高斯模型实现异常检测算法，并将其应用于检测网络上的故障服务器。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.io import loadmat

In [2]:
data=loadmat('/home/mw/input/andrew_ml_ex85033/ex8data1.mat')
X=data['X']
X.shape

(307, 2)

In [3]:
fig,ax=plt.subplots(figsize=(12,8))
ax.scatter(X[:,0],X[:,1])
plt.show()

<Figure size 864x576 with 1 Axes>

In [4]:
def estimate_gaussian(X):
    #mu——X每个维度的平均值
    mu=X.mean(axis=0)
    sigma2=X.var(axis=0)
    #sigma2——X每个维度的方差
    return mu,sigma2

In [5]:
mu,sigma2=estimate_gaussian(X)
mu,sigma2

(array([14.11222578, 14.99771051]), array([1.83263141, 1.70974533]))

In [6]:
xplot=np.linspace(0,25,100)
yplot=np.linspace(0,25,100)
Xplot,Yplot=np.meshgrid(xplot,yplot)
Z=np.exp((-0.5)*((Xplot-mu[0])**2/sigma2[0]+(Yplot-mu[1])**2/sigma2[1]))
fig,ax=plt.subplots(figsize=(12,8))
contour=plt.contour(Xplot,Yplot,Z,[10**-11, 10**-7, 10**-5, 10**-3, 0.1,0.5],colors='r')
ax.clabel(contour, inline=1, fontsize=10)
ax.scatter(X[:,0],X[:,1])
plt.show()

<Figure size 864x576 with 1 Axes>

**以上计算了高斯分布的参数，根据参数即可估计每组数据的概率，低概率的点更有可能是异常的。确定异常点首先需要确定一个阈值，下面通过验证集确定阈值**

In [7]:
Xval=data['Xval']
yval=data['yval']
Xval.shape,yval.shape

((307, 2), (307, 1))

In [8]:
#norm(loc, scale) loc: mean 均值， scale: standard deviation 标准差（注意不是方差）
sigma=np.sqrt(sigma2)
from scipy import stats
dist=stats.norm(mu[0],sigma[0])
#输出概率密度值
dist.pdf(15)

0.23767635105892454

In [9]:
dist.pdf(X[:,0])[0:50]

array([0.21620977, 0.25745208, 0.29413223, 0.24721192, 0.27251547,
       0.2918119 , 0.18713958, 0.15117648, 0.09356331, 0.166609  ,
       0.29338708, 0.29448769, 0.25559237, 0.25595621, 0.2932714 ,
       0.2944456 , 0.29288017, 0.28518331, 0.27727759, 0.09489765,
       0.27027271, 0.29342161, 0.24110555, 0.29304288, 0.19607729,
       0.15652979, 0.27590459, 0.25749622, 0.27667047, 0.2834953 ,
       0.17068283, 0.29318613, 0.20432637, 0.19577297, 0.10897052,
       0.24595126, 0.14063746, 0.29463374, 0.28052751, 0.29371857,
       0.28168158, 0.22524293, 0.28218942, 0.29429662, 0.26674962,
       0.28856013, 0.11137104, 0.29467721, 0.28904196, 0.18556585])

In [10]:
p=np.zeros(X.shape)
p[:,0]=stats.norm(mu[0],sigma[0]).pdf(X[:,0])
p[:,1]=stats.norm(mu[1],sigma[1]).pdf(X[:,1])
p.shape

(307, 2)

In [11]:
pval=np.zeros(Xval.shape)
pval[:,0]=stats.norm(mu[0],sigma[0]).pdf(Xval[:,0])
pval[:,1]=stats.norm(mu[1],sigma[1]).pdf(Xval[:,1])
pval.shape

(307, 2)

**下面实现求阈值的函数**

In [12]:
def select_threshold(pval,yval):
    best_epsilon=0
    #使用F1值
    best_f1=0
    f1=0
    step=(pval.max()-pval.min())/1000
    for epsilon in np.arange(pval.min(),pval.max(),step):
        #小于阈值
        preds=pval<epsilon
        #y的值为1 表示为异常点
        tp=np.sum(np.logical_and(preds==1,yval==1)).astype(float)
        fp=np.sum(np.logical_and(preds==1,yval==0)).astype(float)
        fn=np.sum(np.logical_and(preds==0,yval==1)).astype(float)
        precision=tp/(tp+fp)
        recall=tp/(tp+fn)
        f1=(2*precision*recall)/(precision+recall)
        if f1>best_f1:
            best_f1=f1
            best_epsilon=epsilon
    return best_epsilon,best_f1

In [13]:
epsilon,f1=select_threshold(pval,yval)
epsilon,f1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


(0.0015255071244515167, 0.7142857142857143)

**以上确定了阈值，下面将阈值应用于数据集**

In [14]:
#获取异常点的索引
outliers=np.where(p<epsilon)
outliers

(array([300, 301, 301, 303, 303, 304, 306, 306]),
 array([1, 0, 1, 0, 1, 0, 0, 1]))

In [15]:
fig,ax=plt.subplots(figsize=(12,8))
ax.scatter(X[:,0],X[:,1])
ax.scatter(X[outliers[0],0],X[outliers[0],1],s=50,color='r',marker='X')
plt.show()

<Figure size 864x576 with 1 Axes>

### 二：实现协同过滤推荐算法，并将其应用于电影评分数据集。

In [16]:
data=loadmat('/home/mw/input/andrew_ml_ex85033/ex8_movies.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Thu Dec  1 17:19:26 2011',
 '__version__': '1.0',
 '__globals__': [],
 'Y': array([[5, 4, 0, ..., 5, 0, 0],
        [3, 0, 0, ..., 0, 0, 5],
        [4, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'R': array([[1, 1, 0, ..., 1, 0, 0],
        [1, 0, 0, ..., 0, 0, 1],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}

In [17]:
Y=data['Y']
R=data['R']
Y.shape,R.shape

((1682, 943), (1682, 943))

In [18]:
#以下代码与之等效Y[1].sum(),np.sum(Y[1]>0),420/131
Y[1,np.where(R[1,:]==1)[0]].mean()

3.2061068702290076

**下面将矩阵渲染为图像，从图像中可以获取用户与电影的相对密度**

In [19]:
fig,ax=plt.subplots(figsize=(12,12))
ax.imshow(Y)
ax.set_xlabel('Users')
ax.set_ylabel('Movies')
fig.tight_layout()
plt.show()

<Figure size 864x864 with 1 Axes>

In [20]:
def serialize(X,theta):
    #序列化矩阵
    # X (movie, feature), (1682, 10): 电影特征
    # theta (user, feature), (943, 10): 用户偏好
    return np.concatenate((X.ravel(),theta.ravel()))

In [26]:
def deserialize(param, n_movie, n_user, n_features):
    #逆序列化
    return param[:n_movie*n_features].reshape(n_movie,n_features),param[n_movie*n_features:].reshape(n_user,n_features)

In [27]:
def cost(param,Y,R,n_features):
    # Y (movie, user), (1682, 943): (movie, user) 评分
    # R (movie, user), (1682, 943): (movie, user) 是否已评价
    # X (movie, feature), (1682, 10): 电影特征
    # theta (user, feature), (943, 10): 用户偏好
    n_movie,n_user=Y.shape
    X,theta=deserialize(param, n_movie, n_user, n_features)
    inner=np.multiply(X@theta.T-Y,R)
    return np.power(inner,2).sum()/2

In [28]:
params_data=loadmat('/home/mw/input/andrew_ml_ex85033/ex8_movieParams.mat')
X=params_data['X']
theta=params_data['Theta']
X.shape,theta.shape

((1682, 10), (943, 10))

In [29]:
#为节省时间只取一小块数据
users=4
movies=5
features=3
X_sub = X[:movies, :features]
theta_sub = theta[:users, :features]
Y_sub = Y[:movies, :users]
R_sub = R[:movies, :users]
param_sub=serialize(X_sub, theta_sub)
cost(param_sub,Y_sub,R_sub,features)

22.224603725685675

In [30]:
param=serialize(X,theta)
cost(param,Y,R,10)

27918.64012454421

In [32]:
def gradient(param,Y,R,n_features):
    n_movies,n_user=Y.shape
    X,theta=deserialize(param, n_movie, n_user, n_features)
    inner=np.multiply(X@theta.T-Y,R)
    X_grad=inner@theta
    theta_grad=inner.T@X
    return serialize(X_grad,theta_grad)

In [34]:
n_movie,n_user=Y.shape
X_grad,theta_grad=deserialize(gradient(param,Y,R,10),n_movie,n_user,10)
X_grad,theta_grad

(array([[-6.26184144,  2.45936046, -6.87560329, ..., -4.81611896,
          3.84341521, -1.88786696],
        [-3.80931446,  1.80494255, -2.63877955, ..., -3.55580057,
          2.1709485 ,  2.65129032],
        [-3.13090116,  2.54853961,  0.23884578, ..., -4.18778519,
          3.10538294,  5.47323609],
        ...,
        [-1.04774171,  0.99220776, -0.48920899, ..., -0.75342146,
          0.32607323, -0.89053637],
        [-0.7842118 ,  0.76136861, -1.25614442, ..., -1.05047808,
          1.63905435, -0.14891962],
        [-0.38792295,  1.06425941, -0.34347065, ..., -2.04912884,
          1.37598855,  0.19551671]]),
 array([[-1.54728877,  9.0812347 , -0.6421836 , ..., -3.92035321,
          5.66418748,  1.16465605],
        [-2.58829914,  2.52342335, -1.52402705, ..., -5.46793491,
          5.82479897,  1.8849854 ],
        [ 2.14588899,  2.00889578, -4.32190712, ..., -6.83365682,
          1.78952063,  0.82886788],
        ...,
        [-4.59816821,  3.63958389, -2.52909095, ..., -

**向代价函数与梯度函数中添加正则项**

In [35]:
def regularized_cost(param,Y,R,n_features,l=1):
    reg_term=np.power(param,2).sum()*(l/2)
    return cost(param,Y,R,n_features)+reg_term

In [36]:
def regularized_gradient(param,Y,R,n_features,l=1):
    grad=gradient(param,Y,R,n_features)
    reg_term=l*param
    return grad+reg_term

In [37]:
regularized_cost(param_sub,Y_sub,R_sub,features,l=1.5)

31.34405624427422

In [38]:
regularized_cost(param,Y,R,10,l=1)

32520.682450229557

In [39]:
n_movie,n_user=Y.shape
X_grad,theta_grad=deserialize(regularized_gradient(param,Y,R,10),n_movie,n_user,10)

In [40]:
movie_list=[]
f=open('/home/mw/input/andrew_ml_ex85033/movie_ids.txt',encoding='gbk')
for line in f:
    tokens=line.strip().split(' ')
    movie_list.append(' '.join(tokens[1:]))
movie_list=np.array(movie_list)

In [41]:
movie_list[0]

'Toy Story (1995)'

In [43]:
ratings = np.zeros((1682, 1))

ratings[0] = 4
ratings[6] = 3
ratings[11] = 5
ratings[53] = 4
ratings[63] = 5
ratings[65] = 3
ratings[68] = 5
ratings[97] = 2
ratings[182] = 4
ratings[225] = 5
ratings[354] = 5

ratings.shape

(1682, 1)

In [44]:
Y=data['Y']
Y=np.append(ratings,Y,axis=1)
Y.shape

(1682, 944)

In [47]:
R=data['R']
R=np.append(ratings!=0,R,axis=1)
R.shape

(1682, 944)

In [57]:
movies=Y.shape[0]
users=Y.shape[1]
features=10
learning_rate=10
X=np.random.random(size=(movies,features))
theta=np.random.random(size=(users,features))
params=serialize(X,theta)
X.shape,theta.shape,params.shape

((1682, 10), (944, 10), (26260,))

In [58]:
Y_norm=Y-Y.mean()
Y_norm.mean()

4.6862111343939375e-17

**进行模型的训练**

In [60]:
from scipy.optimize import minimize
fmin=minimize(fun=regularized_cost,x0=params,args=(Y_norm,R,features,learning_rate),method='TNC',jac=regularized_gradient)
fmin

     fun: 69384.2027948376
     jac: array([ 8.73284215e-06,  1.31383006e-05,  1.66515218e-05, ...,
        1.32488889e-06, -1.73045026e-06,  2.28293149e-06])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 1180
     nit: 39
  status: 1
 success: True
       x: array([0.44169541, 0.7123411 , 1.08200719, ..., 0.59930305, 0.12367505,
       0.94595557])

In [62]:
X_trained,theta_trained=deserialize(fmin.x,movies,users,features)
X_trained.shape,theta_trained.shape

((1682, 10), (944, 10))

In [76]:
prediction = X_trained @ theta_trained.T
#对添加的第0个人进行预测
my_preds = prediction[:, 0] + Y.mean()
#print(a[::-1]) 取从后向前（相反）的元素
idx = np.argsort(my_preds)[::-1]  # Descending order
idx.shape

(1682,)

In [80]:
#评分的top10
my_preds[idx][:10]

array([4.24829034, 4.23768158, 4.01220525, 3.99085122, 3.94317845,
       3.92094498, 3.91026429, 3.89219926, 3.81424418, 3.77801846])

In [82]:
#输出评分top10的电影名称
for m in movie_list[idx][:10]:
    print(m)

Star Wars (1977)
Titanic (1997)
Raiders of the Lost Ark (1981)
Return of the Jedi (1983)
Empire Strikes Back, The (1980)
Shawshank Redemption, The (1994)
Good Will Hunting (1997)
Braveheart (1995)
Godfather, The (1972)
Schindler's List (1993)
